Among the following models, which model fits the Pantheon supernovae compilation better? why?
$\Lambda$CDM, Flat $\Lambda$CDM, $w$CDM, Flat $w$CDM



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.cosmology import FlatLambdaCDM, LambdaCDM, wCDM,FlatwCDM
from scipy import optimize
import pandas as pd

# These packages are not installed by default on Google Colab, we may need to
# install them
# I am using try/except so that we only install them if they're not already
# installed:
try:
   import emcee
except:
  # emcee not installed, install it and import
  !pip install emcee
  import emcee
# same for corner:
try:
  import corner
except:
  !pip install corner
  import corner
  
try:
  import dynesty
except:
  !pip install dynesty
  import dynesty


In [ ]:

models = ["FlatLCDM","LCDM",'FlatwCDM','wCDM']

In [ ]:
z, mB, dmB = np.genfromtxt("https://raw.githubusercontent.com/dscolnic/Pantheon/master/lcparam_full_long.txt", skip_header=0, usecols=(1, 4, 5), unpack=True)
Nsn = len(z)
Csys = np.genfromtxt("https://raw.githubusercontent.com/dscolnic/Pantheon/master/sys_full_long.txt", skip_header=1).reshape((Nsn,Nsn))
Cinv = np.linalg.inv(Csys + np.diag(dmB**2))




In [ ]:

# creating a structured array:
data = np.zeros(Nsn,dtype=[('z',float),('mB',float),('dmB',float)])
data['z'] = z
data['mB'] = mB
data['dmB'] = dmB

# creating a dataframe to save my results
df = pd.DataFrame(columns=['Model','AIC','BIC','DIC','chi2min','chi2min/dof'])
df['Model'] = models

In [ ]:
# Define my residual function (cost function):

def residuals(params,model,data):
  '''return residuals

  order: Om0, Ode0, w,Mb
  '''
  assert model in [0,1,2,3], 'error, model={} unknown'.format(model)
  H0 = 70.
  if model == 0:
    Om0,MB = params
    cosmo = FlatLambdaCDM(Om0=Om0,H0=H0)
  elif model == 1:
    # LCDM:
    Om0,Ode0,MB = params
    cosmo = LambdaCDM(Om0=Om0,Ode0=Ode0,H0=H0)
  elif model == 2:
    # flat wCDM
    Om0,w,MB = params
    cosmo = FlatwCDM(Om0=Om0,w0=w,H0=H0)
  elif model == 3:
    Om0,Ode0,w,MB = params
    cosmo = wCDM(Om0=Om0,Ode0=Ode0,w0=w,H0=H0)
  dy = cosmo.distmod(data['z']).value - (data['mB'] - MB )
  chi2 = dy.T @ Cinv @ dy
  return chi2




## 1. Calculate AIC and BIC

Minimize chi2 = Maximum Likelihood

In [ ]:
# minimizing chi2 = max likelihood


Nparams = [2,3,3,4]
bounds = {}
bounds[0] = ((0,1),(-25,-15))
bounds[1] = ((0,1),(0,1),(-25,-15))
bounds[2] = ((0,1),(-2,0),(-25,-15))
bounds[3] = ((0,1),(0,1),(-2,0),(-25,-15))
thetahat = {}
for model in range(4):
  print (72*"=")
  print(" Model: {}".format(model))
  res = optimize.differential_evolution(residuals,bounds=bounds[model],args=(model,data))
  # x0 = np.zeros(Nparams[model])
  # print (x0,bounds[model],)
  # res = optimize.minimize(residuals,x0,args=(model,data),bounds=bounds[model])
  print (res)
  thetahat[model] = res.x
  chi2 = res.fun
  nu = Nsn-Nparams[model]
  print( " chi2/nu = {}/{} = {}".format(chi2,nu,chi2/nu))

  ii = df['Model'] == models[model]
  print (ii,df.loc[ii])
  df.loc[ii,'AIC'] = chi2 + 2*Nparams[model]
  df.loc[ii,'BIC'] = chi2 + Nparams[model] * np.log(Nsn)
  df.loc[ii,'chi2min'] = chi2
  df.loc[ii,'chi2min/dof'] = chi2/nu
  print (df)
  # print (" AIC={}".format(DIC))

print (df)

# Run MCMC to calculate the DIC

In [ ]:
# Define my priors:

def lnpOm0(Om0):
  if 0<Om0<1:
    return 0
  else:
    return -np.inf


def lnpMB(MB):
  ''' Note: this is an unnormalized prior.
  The correct normalization would be -np.log(Mb_Max-Mb_min) = np.log(1/(Mbmax-Mbmin))
  '''
  if -25<MB<-15:
    return 0
  else:
    return -np.inf

def lnpOde0(Ode0):
  if 0<Ode0<1:
    return 0
  else:
    return -np.inf

def lnpw(ww):
  if -2<ww<0:
    return 0
  else:
    return -np.inf

def lnprior(theta,model):
  '''Returns the (unnormalized) prior'''
  Om0 = theta[0]
  MB = theta[-1]
  lp = lnpOm0(Om0) + lnpMB(MB)
  if model in [1,3]:
    Ode0 = theta[1]
    lp += lnpOde0(Ode0)
  if model in [2,3]:
    ww = theta[-2]
    lp += lnpw(ww)
  return lp

def lnP(theta,model,data):
  '''Calculate the log posterior
  ln P(theta) = ln pi(theta) + ln L (theta)
  Also return the log likelihood as a blob.
  '''
  # 1. Get the prior for the parameters theta
  lp = lnprior(theta,model)

  # are the parameters within the priors?
  if np.isfinite (lp):
    # Get the log likelihood
    ll = -.5*residuals(theta,model,data)
    return lp+ll,ll
  else:
    # my prior is zero so I don't need to calculate ln L,
    # I just return ln pi = -inf
    return lp,lp


# Calculate the DIC

In [ ]:

# listDIC = {}
# listDIC[2] =  1029.2951853008806
# listDIC[0] = 1030.5038073377136

def DIC (mysample,thetamean):
  # ln L = -.5*residuals(theta,model,data)
  ll = mysample.get_blobs(flat=True,thin=nthin,discard=nburn).mean()
  # chi2(\bar\theta) = residuals = -2 ln L(\hat\theta)
  return -4*ll  - residuals(thetamean,model,data)


# Ok, now we run everything:

In [ ]:
samplers = {}

# fLCDM
model = 0
nwalker = 12
ndim = Nparams[model]
niter = 2000

theta0 = thetahat[model] + 1e-4 * np.random.randn(nwalker, ndim)
nwalkers, ndim = theta0.shape

samplers[model] = emcee.EnsembleSampler(nwalkers, ndim, lnP, args=(model,data))
samplers[model].run_mcmc(theta0, niter, progress=True);



fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = samplers[model].get_chain()
labels = [r"$\Omega_m$", r"M_B",]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

    ax.axvline(x=50)
axes[-1].set_xlabel("step number");


# AutocorrelatioN:
tau = samplers[model].get_autocorr_time()
print(tau)
nburn = 50
nthin = 30


flat_samples = samplers[model].get_chain(discard=nburn, thin=nthin, flat=True)
print(flat_samples.shape)
thetamean = flat_samples.mean(axis=0)
print(thetamean)

fig = corner.corner(
    flat_samples, labels=labels, truth=thetahat
);


myDIC = DIC(samplers[model],thetamean)
print (myDIC)
df.loc[model,'DIC'] = myDIC
print (df)

In [ ]:

fig = corner.corner(
    flat_samples, labels=labels, truth=thetahat
);

fig.savefig("lcdm.pdf",dpi=600)

In [ ]:
df

In [ ]:

# LCDM
# initialize my MCMC:
model = 1
nwalker = 12
ndim = Nparams[model]
niter = 4000

# initial condition:
theta0 = thetahat[model] + 1e-4 * np.random.randn(nwalker, ndim)
nwalkers, ndim = theta0.shape

samplers[model] = emcee.EnsembleSampler(nwalkers, ndim, lnP, args=(model,data))
samplers[model].run_mcmc(theta0, niter, progress=True);



fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = samplers[model].get_chain()
labels = [r"$\Omega_m$", r"$\Omega_\Lambda$" ,r"$M_B$"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

    ax.axvline(x=50)
axes[-1].set_xlabel("step number");

# AutocorrelatioN:
tau = samplers[model].get_autocorr_time()
print(tau)
nburn = 50 # remove the first nburn iterations
nthin = 30 # remove every nthin iteration to remove the correlation

flat_samples = samplers[model].get_chain(discard=nburn, thin=nthin, flat=True)
print(flat_samples.shape)
thetamean = flat_samples.mean(axis=0)
print(thetamean)

fig = corner.corner(
    flat_samples, labels=labels, truth=thetahat
);


myDIC = DIC(samplers[model],thetamean)
print (myDIC)
df.loc[model,'DIC'] = myDIC
print (df)

In [ ]:

# flatwCDM
model = 2
nwalker = 12
ndim = Nparams[model]
niter = 8000

theta0 = thetahat[model] + 1e-4 * np.random.randn(nwalker, ndim)
nwalkers, ndim = theta0.shape

samplers[model] = emcee.EnsembleSampler(nwalkers, ndim, lnP, args=(model,data))
samplers[model].run_mcmc(theta0, niter, progress=True);



fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = samplers[model].get_chain()
labels = [r"$\Omega_m$", r"$w$" ,r"$M_B$"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

    ax.axvline(x=50)
axes[-1].set_xlabel("step number");

# AutocorrelatioN:
tau = samplers[model].get_autocorr_time()
print(tau)
nburn = 50
nthin = 30

flat_samples = samplers[model].get_chain(discard=nburn, thin=nthin, flat=True)
print(flat_samples.shape)
thetamean = flat_samples.mean(axis=0)
print(thetamean)

fig = corner.corner(
    flat_samples, labels=labels, truth=thetahat
);



myDIC = DIC(samplers[model],thetamean)
print (myDIC)
df.loc[model,'DIC'] = myDIC
print (df)
# myDIC[model] = DIC(sampler,thetamean)
# # {2: 1029.2951853008806}

In [ ]:

# wCDM
model = 3
nwalker = 12
ndim = Nparams[model]
niter = 10000

print (thetahat[model])
theta0 = thetahat[model] + 1e-4 * np.random.randn(nwalker, ndim)
nwalkers, ndim = theta0.shape

samplers[model] = emcee.EnsembleSampler(nwalkers, ndim, lnP, args=(model,data))
samplers[model].run_mcmc(theta0, niter, progress=True);



# # AutocorrelatioN:
# #tau = sampler.get_autocorr_time()
# print(tau)
# nburn = 50
# nthin = 30

# flat_samples = sampler.get_chain(discard=nburn, thin=nthin, flat=True)
# print(flat_samples.shape)
# thetamean = flat_samples.mean(axis=0)
# print(thetamean)

# fig = corner.corner(
#     flat_samples, labels=labels, truth=thetahat
# );


In [ ]:


samples = samplers[model].get_chain()
print (samples.shape)
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
labels = [r"$\Omega_m$", r"$\Omega_\Lambda$", r"$w$" ,r"$M_B$"]
for i,ax in enumerate(axes):
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

    ax.axvline(x=50)
axes[-1].set_xlabel("step number");

In [ ]:

# AutocorrelatioN:
#tau = sampler.get_autocorr_time()
# print(tau)
nburn = 50
nthin = 30

flat_samples = samplers[model].get_chain(discard=nburn, thin=nthin, flat=True)
print(flat_samples.shape)
thetamean = flat_samples.mean(axis=0)
print(thetamean)

fig = corner.corner(
    flat_samples, labels=labels, truth=thetahat
);



myDIC = DIC(samplers[model],thetamean)
print (myDIC)
df.loc[model,'DIC'] = myDIC
print (df)

# Dynesty

In [ ]:
from dynesty import NestedSampler

def loglike(theta,model,data):
    return -.5*residuals(theta,model,data)

def ptform(uu,model):
    '''uu: (0,1)'''
    theta = np.zeros_like(uu)
    ii = 0
    # Om0: 
    theta[ii] = uu[0]
    ii+=1
    if model in [1,3]:
        # theta[1]: Ode0   in (0,1) 
        theta [ii] = uu[ii]
        ii += 1
    if model in [2,3]:
        # theta[2]: ww in [-2,0]
        theta[ii] = 2*uu[ii]-2
    return theta

# initialize our nested sampler
# sampler = NestedSampler(loglike, ptform, ndim)

In [ ]:
# flcdm
imodel = 0
model = models[imodel]
ndim = Nparams[imodel]
sampler = NestedSampler(loglike, ptform, ndim,
                        logl_args=(imodel,data),
                        ptform_args=(imodel,)
                        )

In [ ]:

sampler.run_nested()
sresults = sampler.results

# "Dynamic" nested sampling.
dsampler = dynesty.DynamicNestedSampler(loglike, ptform, ndim)
dsampler.run_nested()
dresults = dsampler.results